<h1><center>Your title </center></h1>

In [ ]:
from datetime import date
print(f'This notebook was prepared on {date.today()}.')

In [ ]:
%reset -sf
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import plotly.express as px
import plotly.io as pio
pio.templates.default = "none"
pd.set_option('display.max_columns', None) #display all dataframe columns

In [ ]:
print("""Please input file location here: 
- Right click csv file
- Select 'Copy as path'
- After pasting, remove double quotation marks at both ends
- Press Enter""")
file_location= input()

In [ ]:
###################################################################
########## PART I: Read in file and tidy up table #################
###################################################################
main_df= pd.read_csv(file_location)
main_df= (main_df
          .query("`Unnamed: 2`== 'Sample'") #selecting samples only
          .drop(['Unnamed: 1','Unnamed: 2','Unnamed: 3'] , axis=1) #dropping columns not needed
          .set_index('Sample'))

#separate dataframe into three different ones: 
concentration_native = main_df.iloc[:,0::3]
area_native = main_df.iloc[:,1::3]
area_IS = main_df.iloc[:,2::3]

concentration_native.columns= (concentration_native.columns
                               .str.replace(' Results', '')
                               .str.replace('-', '_'))

#rename compound names for area_native df:
area_native.columns = concentration_native.columns

#Transpose dataframes
area_native = area_native.T.astype('float64')
concentration_native = concentration_native.T.astype('float64')
area_IS = area_IS.T.astype('float64')

#Remove duplicate IS area values:
unique= [i for idx,i in enumerate(area_IS.index) if len(area_IS.index.str.split('.')[idx]) == 1]
area_IS= area_IS[area_IS.index.isin(unique)]
area_IS.index= area_IS.index.str.replace("Results", '', regex= True)

#user input needed: create a csv where we can input the amounts of AMAP extracted
(pd.Series(area_IS.index)
 .to_csv('IS_table_empty.csv', index = False))

print(f'Please check your folder at {file_location} for a file named: IS_table_empty.csv.')
print(f'Input the amount of internal standards in the samples [in pg].')

In [ ]:
#Read in the file as pd.series:
IS_table= pd.read_csv('IS_table_empty.csv', index_col = 0, squeeze= True, names= None)

In [ ]:
###################################################################
############# PART II: Calculate response factors #################
###################################################################

area_CB_207_in_ISRS= area_native[area_native.columns[area_native.columns.str.contains('IS_RS')]].loc['CB_207',:] #extracting peak CB_207 peak areas
area_IS_in_ISRS= area_IS[area_IS.columns[area_IS.columns.str.contains('IS_RS')]]
#IS_table= pd.Series([1250,12500,1000,2500,2500], index= area_IS.index) #Set concentration values for internal standards
RRF= ((area_IS_in_ISRS * 1000) /  area_CB_207_in_ISRS).div(IS_table, axis= 0) #https://stackoverflow.com/questions/53217607/how-do-i-operate-on-a-dataframe-with-a-series-for-every-column
#AVG_RRF=  RRF.T.apply(lambda x: x.mean())
Statistics_RRF= RRF.T.describe()

fig = px.bar(data_frame=Statistics_RRF.T, y= 'mean', color=Statistics_RRF.T.index,
       error_y= Statistics_RRF.T['std'])

fig.update_layout(
    #height=400,
    legend_title_text='Internal standards',
    title=dict(
        text='<b>Calculated relative response factors <br><sup>Error bars represent standard deviation</sup></b>',
        x=0.5,
        y=0.9,
        font=dict(
            family="Arial",
            size=20,
            color='#000000'
        )
    ),
    xaxis_title='',
    yaxis_title='<b>RRF</b>',
    font=dict(
        family="Arial",
        size=12,
        color='#000000'
    )
)
fig.update_layout(
    margin=dict(l=100, r=100, t=100, b=100)
)
fig.write_html("RRF.html")
fig

In [ ]:
###################################################################
# PART IIIa: Calculate the amount of internal standard extracted ##
###################################################################
#extract CB-207 areas from all samples except IS_RS
area_CB_207_without_ISRS= area_native[area_native.columns[~area_native.columns.str.contains('IS_RS')]].loc['CB_207',:] 
#extract IS areas from all samples except IS_RS
area_IS_without_ISRS= area_IS[area_IS.columns[~area_IS.columns.str.contains('IS_RS')]]
#calculate amount of internal standard extracted
amount_IS_extracted= (area_IS_without_ISRS * 1000 / area_CB_207_without_ISRS).div(Statistics_RRF.T['mean'], axis= 0)

###################################################################
########## PART IIIb: Calculate extraction efficiencies ###########
###################################################################

extraction_efficiency= amount_IS_extracted.div(IS_table, axis= 0) * 100
Statistics_ExtractionEfficiency= extraction_efficiency.T.describe()

#reshape table so that it can be plotted nicely
extraction_efficiency_plt= pd.melt(frame= extraction_efficiency.T, ignore_index= False, 
        var_name= 'ISTD', value_name= 'Extraction_efficiency')

fig = px.box(extraction_efficiency_plt, x= 'ISTD', y= 'Extraction_efficiency', color= 'ISTD', points= 'all',
            hover_name=extraction_efficiency_plt.index, range_y= [0, extraction_efficiency_plt.Extraction_efficiency.max() + 10])
fig.update_layout(
    #height=400,
    legend_title_text='Internal standards',
    title=dict(
        text='<b>Calculated extraction efficiencies</b>',
        x=0.5,
        y=0.9,
        font=dict(
            family="Arial",
            size=20,
            color='#000000'
        )
    ),
    xaxis_title='',
    yaxis_title='<b>EE %</b>',
    font=dict(
        family="Arial",
        size=12,
        color='#000000'
    )
)
fig.update_layout(
    margin=dict(l=100, r=100, t=100, b=100)
)
fig.write_html("EE.html")
fig

In [ ]:
#################################################################################
##### PART IV: Calculate correction factors based on AMAP concentrations ########
#################################################################################

#extract native concentration values in blanks and AMAPs
concentration_native_in_blanks= concentration_native[concentration_native.columns[concentration_native.columns.str.contains('Blank|blank', regex= True)]]
concentration_native_in_AMAP= concentration_native[concentration_native.columns[concentration_native.columns.str.contains('AMAP|amap', regex= True)]]

#calculate the native concentration means in the blanks 
AVG_concentration_native_in_blanks= concentration_native_in_blanks.mean(axis= 1)

#user input needed: create a csv where we can input the amounts of AMAP extracted
(pd.Series(concentration_native_in_AMAP.columns)
 .to_csv('AMAP_SampleVolume_empty.csv', index = False))

print(f'Please check your folder at {file_location} for a file named: AMAP_SampleVolume_empty.csv.')
print(f'Input the amount of AMAP samples extracted in ml.')

In [ ]:
print("""Please input file location here: 
- Right click csv file
- Select 'Copy as path'
- After pasting, remove double quotation marks at both ends
- Press Enter""")
AMAP_SampleVolume= input()

In [ ]:
#Read in the file as pd.series:
AMAP_SampleVolume= pd.read_csv(AMAP_SampleVolume, index_col = 0, squeeze= True)

In [ ]:
#calculate the native concentrations observed in the AMAP samples in ng/ml
AMAP_concentration_ngml= (concentration_native_in_AMAP.sub(AVG_concentration_native_in_blanks, axis= 0)
.div(AMAP_SampleVolume, axis= 1)) / 1000

#calculate the average concentrations of native analytes in the AMAPs
AVG_AMAP_concentration_ngml= AMAP_concentration_ngml.mean(axis= 1)

#user input needed: create a csv where we can input the concentrations of native analytes in the AMAPs
(pd.Series(AVG_AMAP_concentration_ngml.index)
 .to_csv('AMAP_concentration_empty.csv', index = False))

print(f'Please check your folder at {file_location} for a file named: AMAP_concentration_empty.csv.')
print(f'Input the concentration of analytes present in the reference AMAP sample in ng/ml.')

In [ ]:
print("""Please input file location here: 
- Right click csv file
- Select 'Copy as path'
- After pasting, remove double quotation marks at both ends
- Press Enter""")
AMAP_theroretical_concentration= input()

In [ ]:
#Read in the file as pd.series:
AMAP_theroretical_concentration= pd.read_csv(AMAP_theroretical_concentration, index_col = 0, squeeze= True)

In [ ]:
#get the correction factors by dividing the theoretical with the measured AVG, replace values with 1
#where we get a 0 or negative value
correction_factors= pd.Series(np.where(AMAP_theroretical_concentration / AVG_AMAP_concentration_ngml > 0, 
                                       AMAP_theroretical_concentration / AVG_AMAP_concentration_ngml, 1), 
                              index= AMAP_theroretical_concentration.index)

fig = px.bar(data_frame= correction_factors.sort_values(), y= correction_factors.sort_values().values)
fig.update_layout(
    #height=400,
    legend_title_text='Internal standards',
    title=dict(
        text='<b>Calculated correction factors</b>',
        x=0.5,
        y=1,
        font=dict(
            family="Arial",
            size=20,
            color='#000000'
        )
    ),
    xaxis_title='',
    yaxis_title='<b>Correction factor (assigned value/measured) </b>',
    font=dict(
        family="Arial",
        size=12,
        color='#000000'
    )
)
fig.add_hline(y=1, line_width=5, line_dash="dash", line_color="red")
fig.write_html("correction_factor.html")
fig

In [ ]:
#################################################################################
########### PART V: Calculate native concentrations in samples ##################
#################################################################################
sample_concentrations= (concentration_native[concentration_native.
                                        columns[~concentration_native.columns.
                                                str.contains('Blank|blank|IS_RS|IS-RS|ISRS|AMAP', regex= True)]])

#user input needed: create a csv where we can input the amounts of sample extracted
(pd.Series(sample_concentrations.columns)
 .to_csv('ExtractedSampleVolume_empty.csv', index = False))

print(f'Please check your folder at {file_location} for a file named: ExtractedSampleVolume_empty.csv.')
print(f'Input the amounts of samples extracted in ml.')

In [ ]:
print("""Please input file location here: 
- Right click csv file
- Select 'Copy as path'
- After pasting, remove double quotation marks at both ends
- Press Enter""")
ExtractedSampleVolume= input()

In [ ]:
#Read in the file as pd.series:
ExtractedSampleVolume= pd.read_csv(ExtractedSampleVolume, index_col = 0, squeeze= True)

In [ ]:
#Calculate the peak areas in the samples
result_sample_concentration= (sample_concentrations
                          .sub(AVG_concentration_native_in_blanks, axis= 0)
                          .mul(correction_factors, axis= 0)
                          .div(ExtractedSampleVolume, axis= 1))

#replace negative values with zero
result_sample_concentration= pd.DataFrame(np.where(result_sample_concentration > 0, result_sample_concentration, 0), 
                                      index= result_sample_concentration.index, columns= result_sample_concentration.columns)

#reshape table so that it can be plotted nicely
result_sample_concentration_melt= pd.melt(frame= result_sample_concentration, ignore_index= False, 
                                      var_name= 'Sample', value_name= 'Peak_area')

fig = px.box(data_frame= result_sample_concentration_melt, x= 'Sample', y= 'Peak_area', color= 'Sample',
            hover_name= result_sample_concentration_melt.index)
fig.update_layout(
    #height=400,
    legend_title_text='Sample Names',
    title=dict(
        text='<b>Calculated concentrations</b>',
        x=0.5,
        y=1,
        font=dict(
            family="Arial",
            size=20,
            color='#000000'
        )
    ),
    xaxis_title='',
    yaxis_title='<b>Calculated concentrations [pg/ml] </b>',
    font=dict(
        family="Arial",
        size=12,
        color='#000000'
    )
)
fig.write_html("Concentration_Samples.html")
fig

In [ ]:
#reshape table so that it can be plotted nicely
result_sample_concentration_melt2= pd.melt(frame= result_sample_concentration.T, ignore_index= False, 
                                       var_name= 'Analyte', value_name= 'Peak_area')


fig = px.box(data_frame= result_sample_concentration_melt2, x= 'Analyte', y= 'Peak_area', color= 'Analyte',
            hover_name= result_sample_concentration_melt2.index)
fig.update_layout(
    #height=400,
    legend_title_text='Analyte Names',
    title=dict(
        text='<b>Calculated concentrations</b>',
        x=0.5,
        y=1,
        font=dict(
            family="Arial",
            size=20,
            color='#000000'
        )
    ),
    xaxis_title='',
    yaxis_title='<b>Calculated concentrations [pg/ml] </b>',
    font=dict(
        family="Arial",
        size=12,
        color='#000000'
    )
)
fig.write_html("Concentration_Analytes.html")
fig

In [ ]:
result_sample_concentration.to_csv('result_sample_concentration.csv')